# 19. RNN with Hamlet

In [1]:
import torch
import torch.nn as nn

import nltk

import random
import numpy as np

## 19.1 Prepare Data

In [2]:
nltk.download("gutenberg")

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\slcf\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [3]:
raw = nltk.corpus.gutenberg.raw("shakespeare-hamlet.txt")
print(raw[:500])

[The Tragedie of Hamlet by William Shakespeare 1599]


Actus Primus. Scoena Prima.

Enter Barnardo and Francisco two Centinels.

  Barnardo. Who's there?
  Fran. Nay answer me: Stand & vnfold
your selfe

   Bar. Long liue the King

   Fran. Barnardo?
  Bar. He

   Fran. You come most carefully vpon your houre

   Bar. 'Tis now strook twelue, get thee to bed Francisco

   Fran. For this releefe much thankes: 'Tis bitter cold,
And I am sicke at heart

   Barn. Haue you had quiet Guard?
  Fran. Not


## 19.2 Char to Dic

In [4]:
char2index = {}
index2char = []

In [5]:
for char in raw :
    if char not in char2index.keys() :
        char2index[char] = len(char2index)
        index2char.append(char)

In [6]:
char2index

{'\n': 27,
 ' ': 4,
 '!': 64,
 '&': 43,
 "'": 39,
 '(': 61,
 ')': 62,
 ',': 48,
 '-': 54,
 '.': 32,
 '1': 23,
 '5': 24,
 '9': 25,
 ':': 42,
 ';': 58,
 '?': 40,
 'A': 28,
 'B': 35,
 'C': 38,
 'D': 55,
 'E': 34,
 'F': 36,
 'G': 51,
 'H': 12,
 'I': 49,
 'K': 46,
 'L': 45,
 'M': 52,
 'N': 41,
 'O': 56,
 'P': 31,
 'Q': 59,
 'R': 53,
 'S': 19,
 'T': 1,
 'V': 63,
 'W': 18,
 'Y': 47,
 'Z': 66,
 '[': 0,
 ']': 26,
 'a': 6,
 'b': 16,
 'c': 29,
 'd': 8,
 'e': 3,
 'f': 11,
 'g': 7,
 'h': 2,
 'i': 9,
 'j': 65,
 'k': 20,
 'l': 14,
 'm': 13,
 'n': 33,
 'o': 10,
 'p': 22,
 'q': 50,
 'r': 5,
 's': 21,
 't': 15,
 'u': 30,
 'v': 44,
 'w': 37,
 'x': 57,
 'y': 17,
 'z': 60}

In [7]:
dim = len(char2index)
print("Dimension of input & output :", dim)

Dimension of input & output : 67


In [8]:
char2vec = {}
eye = np.eye(len(char2index))

In [9]:
for item in char2index.keys() :
    char2vec[item] = eye[char2index[item],:]

In [10]:
char2vec['a']

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [11]:
data = np.array([char2vec[char] for char in raw])

## 19.3 Define Model

In [12]:
# https://pytorch.org/docs/stable/nn.html#torch.nn.RNN

In [13]:
# Parameters:	
# input_size – The number of expected features in the input x
# hidden_size – The number of features in the hidden state h
# num_layers – Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two RNNs together to form a stacked RNN, with the second RNN taking in outputs of the first RNN and computing the final results. Default: 1
# nonlinearity – The non-linearity to use. Can be either ‘tanh’ or ‘relu’. Default: ‘tanh’
# bias – If False, then the layer does not use bias weights b_ih and b_hh. Default: True
# batch_first – If True, then the input and output tensors are provided as (batch, seq, feature). Default: False
# dropout – If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
# bidirectional – If True, becomes a bidirectional RNN. Default: False

In [14]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        self.rnn = nn.RNN(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        out, hidden = self.rnn(input, hidden)
        fc_out = self.fc(out)
        return fc_out, hidden

    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, 1, self.hidden_size).cuda()
        return hidden

In [15]:
model = RNN(dim, 500, dim, 1).cuda()

## 19.4 Train Model

In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss()

In [17]:
seq_len = 100
num_epochs = 5

In [18]:
for epoch in range(num_epochs):
    
    sp = list(range(0, len(data) - 2 * seq_len, seq_len))
    sp = np.add(sp, random.randint(0, seq_len))
    random.shuffle(sp)
    
    for i in range(len(sp)) :
    
        hidden = model.init_hidden()

        cost = 0
            
        for pos in range(sp[i], sp[i] + seq_len):
            
            X = torch.from_numpy(data[pos]).type(torch.FloatTensor).cuda().reshape(1, 1, dim)
            y = torch.from_numpy(data[pos+1]).cuda()
            
            _, y = y.max(dim=0)
            
            pre, hidden = model(X,hidden)
            cost += loss(pre.reshape(1, dim), y.reshape(1).cuda())

        optimizer.zero_grad()
        cost.backward()
        
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        
        optimizer.step()

        if (i + 1) % 100 == 0 :
            print('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f'
                     %(epoch+1, num_epochs, i + 1, len(sp), cost.item()))

Epoch [1/5], Iter [100/1627] Loss: 329.5282
Epoch [1/5], Iter [200/1627] Loss: 313.7830
Epoch [1/5], Iter [300/1627] Loss: 263.7442
Epoch [1/5], Iter [400/1627] Loss: 259.2927
Epoch [1/5], Iter [500/1627] Loss: 258.7591
Epoch [1/5], Iter [600/1627] Loss: 210.3250
Epoch [1/5], Iter [700/1627] Loss: 232.6843
Epoch [1/5], Iter [800/1627] Loss: 262.0320
Epoch [1/5], Iter [900/1627] Loss: 225.5680
Epoch [1/5], Iter [1000/1627] Loss: 234.9646
Epoch [1/5], Iter [1100/1627] Loss: 251.1961
Epoch [1/5], Iter [1200/1627] Loss: 252.1086
Epoch [1/5], Iter [1300/1627] Loss: 206.3881
Epoch [1/5], Iter [1400/1627] Loss: 170.9088
Epoch [1/5], Iter [1500/1627] Loss: 248.8352
Epoch [1/5], Iter [1600/1627] Loss: 237.9035
Epoch [2/5], Iter [100/1627] Loss: 211.3038
Epoch [2/5], Iter [200/1627] Loss: 230.0274
Epoch [2/5], Iter [300/1627] Loss: 210.6045
Epoch [2/5], Iter [400/1627] Loss: 245.9961
Epoch [2/5], Iter [500/1627] Loss: 208.2122
Epoch [2/5], Iter [600/1627] Loss: 193.6741
Epoch [2/5], Iter [700/16

## 19.5 Test Model

In [19]:
start_num = 5
text = index2char[start_num]

model.eval()
hidden = model.init_hidden()

X_test = torch.from_numpy(data[start_num]).type(torch.FloatTensor).cuda().reshape(1, 1, dim)
    
for i in range(500) :

    pre, hidden = model(X_test, hidden)

    temp = pre.reshape(-1).cpu().data.numpy()

    best_5 = np.argsort(temp)[::-1][:5]
    
    temp = np.exp(temp[best_5])
    
    temp = temp / temp.sum()
    
    pre = np.random.choice(best_5, 1, p = temp)[0]
    
    curr_char = index2char[pre]
    
    text += curr_char
    
    X_test = torch.from_numpy(char2vec[curr_char]).type(torch.FloatTensor).cuda().reshape(1, 1, dim)
    
print("* Generated Text : \n\n", text)

* Generated Text : 

 rhis and than the King of your the Sondersone then
I sour to the framation

   Obh. It the be a pertoue,
With shour for a doofe then

  Han. I all with'r hest nene the King. Growes

   Rang. To the fron trent ther frome, the verit whinge and to the King of Horound.

Exhur to to to the Connten,
We dow to to's mode in the the to hant

   Ham. An shat soue for tre fortirn will thon ter this and fron trentent the thoue a frele of a seruld my Lord to bus in the Sonning to might

   Hor. Sin to't and a
